# MOLECULAR DOCKING

<p style="font-size:1.5em;">Written by: ROPÓN-PALACIOS G.<sup>1</sup> y OLIVOS-RAMIREZ G.<sup>2</sup></p>
<p style="font-size:1.0em;"><sup>1</sup>Laboratorio de Canales Iónicos, IMSATED, Universidad Nacional Santiago del Estero, SE, Argentina.<br><sup>2</sup>Facultad de Salud Pública, Universidad Peruana Cayetano Heredia, Lima, Perú.</p>

### Introduction

<p style="text-align: justify;font-family: Arial;color:black;font-size:1.1em;">El enfoque de docking molecular puede usarse para modelar la interacción entre una molécula pequeña y una proteína a nivel atómico, lo que nos permite caracterizar el comportamiento de las moléculas pequeñas en el sitio de unión de las proteínas diana, así como dilucidar procesos bioquímicos fundamentales. El proceso de acoplamiento implica dos pasos básicos: la predicción de la conformación del ligando, así como su posición y orientación dentro de estos sitios (generalmente denominada pose) y la evaluación de la afinidad de unión (energia de interacción = ΔG). Estos dos pasos son ejecutados por algoritmos semiempíricos, uno de busqueda y una función de scoring. Es estudio del mecanismo de unión ligando-receptor se basa en la teoría de inhibición, en la cual el ligando encaja en el receptor bloqueando al ligando original.</p>   
<p style="text-align: justify;font-family: Arial;color:black;font-size:1.1em;">Conocer la ubicación del sitio de enlace antes de los procesos de acoplamiento aumenta significativamente la eficiencia de acoplamiento. En muchos casos, el sitio de unión es conocido antes de acoplar ligandos en él. Además, se puede obtener información sobre los sitios mediante la comparación de la proteína objetivo con una familia de proteínas que comparten una función similar o con proteínas cocristalizadas con otros ligandos. En ausencia de conocimiento sobre los sitios de unión, los programas de detección de cavidades o los servidores en línea (RaptorX,   ), se pueden utilizar para identificar supuestos sitios activos dentro de las proteínas. El acoplamiento sin ningún supuesto sobre el sitio de unión se denomina acoplamiento ciego.</p>
<p style="text-align: justify;font-family: Arial;color:black;font-size:1.1em;">Teniendo en cuenta la limitación de los recursos informáticos, el acoplamiento se realiza de forma ligando flexible y un receptor rígido, siendo el método más popular en uso. Recientemente se han hecho muchos esfuerzos para tratar la flexibilidad del receptor, sin embargo, el acoplamiento flexible del receptor, especialmente la flexibilidad del esqueleto en los receptores, todavía presenta un desafío importante para los métodos de acoplamiento disponibles.</p>

![alt text](docking.png "Title")
<p style="text-align: center;font-family: Arial;color:black;font-size:1.2em;">Figura 1. Interacción proteína-ligando (dihidrofolato reductasa-trimetropin).</p>


## Case of study

<p style="text-align: justify;font-family: Arial;color:black;font-size:1.2em;">Mediante una reconstrucción metabólica basada en el genoma de Spirulina platensis en el servidor ANTISMASH (https://antismash.secondarymetabolites.org/) se identificó que esta microalga
sintetiza un antibiótico conocido, 5’hidroxystroptimicina. Se desea conocer la forma de inhibición de este antibiótico contra un target de Staphylococcus aureus, dihidrofolato reductasa (DHFR). Esta enzima resulta un buen target dado que se encuentra relacionada con los procesos de síntesis de precursores para el metabolismo de las guaninas en la bacteria. Para ello, se realizará un docking molecular del tipo ligando-proteína.</p>


### Step 1: Ligand preparation

<p style="text-align: justify;font-family: Arial;color:black;font-size:1.2em;">El ligando obtenido en el servidor ANTISMASH proporciona el ID del ligando en el repositorio PUBCHEM (ID: 23267). El ligando se descarga en formato SDF con coordenadas en 2D, por lo que será necesario transformarlo a formato PDB, para lo cual utilizaremos BABEL. Posteriormente, realizaremos una conversión de PDB a PDBQT, seguido de una procedimiento de minimización de energía para optimizar la geometría del ligando.</p>

In [ ]:
%%bash
pip install PubChemPy

In [ ]:
from pubchempy import *
download('SDF', 'ligand.sdf', 'Hydroxystreptomycin', 'name')

In [ ]:
%%bash
pymol='/home/gustavo/Instaladores/pymol/pymol/pymol'
$pymol ligand.sdf

In [ ]:
%%bash
babel -isdf ligand.sdf -opdb ligand.pdb --gen3D -p 7.4

In [ ]:
%%bash
pymol='/home/gustavo/Instaladores/pymol/pymol/pymol'
$pymol ligand.pdb

In [ ]:
%%bash 
babel -ipdb ligand.pdb -opdbqt ligand.pdbqt

In [ ]:
%%bash
obminimize -ff MMFF94 -n 5000 -sd -c 1e-3 ligand.pdbqt

In [ ]:
%%bash
pymol='/home/gustavo/Instaladores/pymol/pymol/pymol'
$pymol ligand.pdbqt

### Step 2: Receptor preparation

<p style="text-align: justify;font-family: Arial;color:black;font-size:1.2em;">En esta parte descargaremos el receptor del Protein Data Bank (https://www.rcsb.org/), código 2W9G, en formato PDB. Seguidamente, reparamos el archivo PDB con la herramienta pdbfixer, la cual repara residuos y atomos perdidos, transforma resiudos no estandar y elimina los heteroátomos de la estructura. Por conveniencia, renombramos a nuestro target con el nombre de receptor.pdb. Con el software Autodock tools removeremos el solvente (moleculas de agua), adicionaremos cargas e hidrogenos polares a la estructura, para finalmente guardarla en un archivo de salida PDBQT.</p> 

In [ ]:
%%bash
receptor=2W9G
wget https://files.rcsb.org/download/$receptor.pdb

In [ ]:
%%bash
pymol='/home/gustavo/Instaladores/pymol/pymol/pymol'
$pymol 2W9G.pdb

In [ ]:
%%bash
conda install -c omnia pdbfixer

In [ ]:
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
fixer = PDBFixer(filename='2W9G.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
PDBFile.writeFile(fixer.topology, fixer.positions, open('receptor.pdb', 'w')) 

In [ ]:
%%bash
adt='/home/gustavo/MGLTools-1.5.6/bin/adt'
$adt receptor.pdb

### Step 3: Running Autodock4

<p style="text-align: justify;font-family: Arial;color:black;font-size:1.2em;">El sitio activo de la enzima DHFR ha sido identificado en un trabajo previo realizado por Kobayashi et al. (2014). Los aminoácidos que conforman el sitio activos son: Val 6, Ala 7, Leu 20, Pro 25, Asp 27, Leu28, Val31, Ser 49, Ile 50, Arg 57, Phe 92, y Thr 111. Esa informacion es de utilidad para construir la caja de simulación (grid/box) e indicar las coordenadas (x, y, z) en donde se realizara el docking. Previo al docking, generaremos un archivo que contenga esta informacion (.gpf = grid parameters file) y otro archivo indicando los parametros del docking (.dpf = docking parameters file). Los resultados de las energias de interaccion son mostradas en un archivo .dlg</p> 

In [ ]:
%%bash
prepare_gpf4='/home/gustavo/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/./prepare_gpf4.py'
$prepare_gpf4 -l ligand.pdbqt -r receptor.pdbqt -p gridcenter='-7.833,-29.806,3.750' -p npts='60,60,60' -o grid_map.gpf

In [ ]:
%%bash
autogrid4 -p grid_map.gpf -l grid_map.glg

In [ ]:
%%bash
prepare_dpf4='/home/gustavo/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/./prepare_dpf4.py'
$prepare_dpf4 -l ligand.pdbqt -r receptor.pdbqt -p ga_run=5 -p ga_pop_size=300 -o genetics_algorithm.dpf 

In [ ]:
%%bash
autodock4 -p genetics_algorithm.dpf -l genetics_algorithm.dlg

Finalmente realizaremos una discusión de nuestros resultados.